In [3]:
#import das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#lendo a planilha e transformando em um DataFrame
dataframe = pd.read_csv('brazil_municipalities_population.csv')

In [ ]:
#Verificação de quantidade de linha e quantidade de colunas
dataframe.shape

In [ ]:
#Verificando o nome das colunas existentes
dataframe.columns

In [ ]:
#Exibindo as 5 primeiras linhas do DataFrame
dataframe.head()

In [ ]:
#Contando a quantidade de nan por coluna
dataframe.isna().sum()

In [ ]:
#formatanto quantas casas decimais irá mostrar
pd.set_option('display.float_format', '{:.2f}'.format)
#Descrição da coluna população
dataframe['population'].describe()

In [ ]:
media_populacao = dataframe['population'].mean()
moda_populacao = dataframe['population'].mode()
mediana_populacao = dataframe['population'].median()
desvio_padrao_populacao = dataframe['population'].std()
variancia_populacao = dataframe['population'].var()
print(f'Média da populção de 1970 - 2021: {media_populacao:.2f}')
print(f'Mediana da populção de 1970 - 2021: {mediana_populacao}')
print(f'Desvio padrão da população: {desvio_padrao_populacao:.2f}')
print(f'Variancia da população: {variancia_populacao:.2f}')
print(f'Moda: {moda_populacao}')

In [ ]:
#total de estados
quantidade_estado = len(dataframe['abbrev_state'].unique())
print(f'O Brasil tem {quantidade_estado} estados')

In [ ]:
#Quantidade de cidade por estado e quantidade de cidade total no Brasil
quantidade_de_cidade_estado = dataframe.groupby('abbrev_state')['name_municipality'].nunique().reset_index().sort_values(by='name_municipality', ascending=False)
quantidade_de_cidade_estado.columns = ['Estado', 'Quantidade de cidade']
quantidade_total_brasil = quantidade_de_cidade_estado['Quantidade de cidade'].sum()
print(f'O Brasil tem {quantidade_total_brasil} cidades no total')
for i in range(len(quantidade_de_cidade_estado)):
    print(f'O estado {quantidade_de_cidade_estado.iloc[i]["Estado"]} tem {quantidade_de_cidade_estado.iloc[i]["Quantidade de cidade"]} cidades')

In [ ]:
#Quantidade da populão alinhada por estado e ano
df_populacao_estado_ano = dataframe.groupby(['year', 'abbrev_state'])['population'].sum().reset_index().sort_values(by=['year', 'population'], ascending=[True, False])
df_populacao_estado_ano



In [ ]:
# Lista de cores em hexadecimal (27 cores)
colors = [
    '#FF0000',  # Vermelho
    '#00FF00',  # Verde
    '#0000FF',  # Azul
    '#FFFF00',  # Amarelo
    '#FF00FF',  # Magenta
    '#00FFFF',  # Ciano
    '#800000',  # Marrom
    '#008000',  # Verde escuro
    '#000080',  # Azul marinho
    '#808000',  # Verde oliva
    '#800080',  # Roxo
    '#008080',  # Verde-água
    '#FF8080',  # Rosa claro
    '#80FF80',  # Verde claro
    '#8080FF',  # Azul claro
    '#FF8000',  # Laranja
    '#8000FF',  # Roxo forte
    '#00FF80',  # Verde menta
    '#808000',  # Mostarda
    '#FF0080',  # Rosa choque
    '#0080FF',  # Azul celeste
    '#80FF00',  # Limão
    '#800080',  # Púrpura
    '#FF80FF',  # Rosa claro
    '#00FF00',  # Verde fluorescente
    '#008080',  # Azul petróleo
    '#FF0000'   # Vermelho forte
]

# Criando um dicionário para mapear cada estado a uma cor específica
estado_para_cor = {estado: colors[i % len(colors)] for i, estado in enumerate(df_populacao_estado_ano['abbrev_state'].unique())}

# Configurando o tamanho do gráfico
plt.figure(figsize=(14, 6))

# Plotando o gráfico de linhas para cada estado com cores diferentes
for estado in df_populacao_estado_ano['abbrev_state'].unique():
    df_estado = df_populacao_estado_ano[df_populacao_estado_ano['abbrev_state'] == estado]
    plt.plot(df_estado['year'], df_estado['population'] / 1e6, marker='o', label=estado, color=estado_para_cor[estado])  # Usando a cor do estado

# Customizações do gráfico
plt.title('Crescimento da População de 1970 - 2021 por estado')
plt.xlabel('Ano')
plt.ylabel('População (em milhões)')
plt.legend(title='Estado')
plt.grid(True)

# Exibindo o gráfico
plt.show()